In [3]:
# test to check if ollama is working
import ollama
response = ollama.chat(model='aisingapore/Llama-SEA-LION-v3.5-8B-R', messages=
[
  { 'role': 'user',  'content': "Describe Singapore"}
])
print(response['message']['content'])

Singapore is a small, bustling island nation located at the southern tip of the Malay Peninsula in Southeast Asia. It’s a fascinating blend of modernity and tradition, with a unique history shaped by its strategic location.

Here are some key aspects that define Singapore:

1. **Geography & Climate**: 
   - A city-state comprising one main island (Singapore) and 63 smaller islets.
   - Tropical climate with high humidity and consistent rainfall throughout the year.
   - Known for being a "garden city," with extensive greenery integrated into urban landscapes.

2. **History**:
   - Originally inhabited by indigenous Orang Laut and later Malay fishermen.
   - Became part of various colonial powers (British) in 1819, evolving from a trading post to a key regional hub.
   - Gained independence in 1965 and transformed rapidly through pragmatic governance into the modern nation it is today.

3. **Economy**:
   - A highly developed global financial center with a strong emphasis on trade, fina

In [ ]:
# import libraries and api keys
import time
import os
import requests
from dotenv import load_dotenv
import ollama


load_dotenv()

TOKEN = os.getenv("telegram_api_key")

BASE_URL = f"https://api.telegram.org/bot{TOKEN}/"

In [2]:
# Select local Ollama model (ollama list)
OLLAMA_MODEL = "aisingapore/Llama-SEA-LION-v3.5-8B-R"  
OLLAMA_URL = "http://localhost:11434/api/chat"

In [3]:
# function to send response once ready
def send_message(chat_id, text):
    r = requests.post(BASE_URL + "sendMessage", json={
        "chat_id": chat_id,
        "text": text
    })
    if not r.ok:
        print("sendMessage failed:", r.text)

In [4]:
# function the get the new user messages with offset setting to get new msg only
def get_updates(offset=None):
    params = {"timeout": 30}
    if offset:
        params["offset"] = offset
    r = requests.get(BASE_URL + "getUpdates", params=params, timeout=40)
    
    if not r.ok:
        print("getUpdates error:", r.text)
        return []
    return r.json().get("result", [])

In [ ]:
# Function to get model response 
def get_model_response(user_input):
    try:
        r = requests.post(OLLAMA_URL, json={
            "model": OLLAMA_MODEL,
            "messages": [{"role": "user", "content": user_input}],
            "stream": False})

        r.raise_for_status()
        return r.json()["message"]["content"]
        
    except Exception as e:
        return f"[Error from model: {e}]"

: 

In [ ]:
# keeep bot in a while loop
print("Bot is running... (Ctrl+C to stop)")
last_update_id = None

try:
    while True:
        updates = get_updates(offset=last_update_id + 1 if last_update_id else None)

        for update in updates:
            last_update_id = update["update_id"]

            message = update.get("message", {})
            chat_id = message.get("chat", {}).get("id")
            user_text = message.get("text")

            if not chat_id or not user_text:
                continue  # Skip non-text messages

            print(f"\nIncoming message from {chat_id}: {user_text}")

            if user_text.lower().strip() == "exit":
                send_message(chat_id, "Goodbye! Thank you for chatting.")
                continue

            send_message(chat_id, "🤖 Thinking...")
            reply = get_model_response(user_text)
            send_message(chat_id, reply)

        time.sleep(1)  # Avoid hammering the Telegram API
except KeyboardInterrupt:
    print("\nBot stopped by user.")


Bot is running... (Ctrl+C to stop)

Incoming message from 8175779086: describe Singapore in Singlish
